# Config

In [1]:
plant = "T.aestivum"
input_genome_name = "GCF_018294505.1_IWGSC_CS_RefSeq_v2.1_genomic.fna"

experiment_dir = "./Experiment"
experiment = f"{plant}/positive"

input_genome_path = f'{experiment_dir}/{plant}/{input_genome_name}'

temp_path = f"{experiment_dir}/{experiment}/Temp"
result_path = f"{experiment_dir}/{experiment}/Result"

In [2]:
import os
if(not os.path.exists(f'{experiment_dir}/{experiment}')):
    os.mkdir(f'{experiment_dir}/{experiment}')

# Common

In [3]:
import json
import time
from subprocess import Popen, PIPE, STDOUT
import math
import numpy as np
import pandas as pd
import hashlib
import requests
import os, sys, subprocess
from tqdm.contrib.concurrent import process_map
from tqdm.notebook import tqdm
tqdm.pandas()
import multiprocessing as mp
import shutil
import urllib.parse
import glob
import os
import sys
import networkx
from networkx.algorithms.clique import find_cliques as maximal_cliques
from ast import literal_eval
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
sys.path.append("./src/")
from ct_analizer import get_row
from filter import filter_run
from utils import adjust, bracket_to_ct, fasta_to_df, df_to_fasta,reformat, reformatCT,get_ct_data, ct2dot_bracket, is_nested

In [4]:
if(not os.path.exists(temp_path)):
    os.mkdir(temp_path)
    
if(not os.path.exists(result_path)):
    os.mkdir(result_path)
    
current_path = os.getcwd()

In [5]:
def bracket_row(row):    
    s = row['data']
    index = min(s.find('.'), s.find('('))
    data = row['data']
    row['data'] = data[0:index]
    row['bracket'] = data[index:]
    return row

In [6]:
def isChar(c):
    return not c in [' ', '-', '|']

def adjust(text,n=7):
    text = str(text)    
    return " " * (n - len(text)) + text

def isMature(c1, c2):
    if(c1.isupper() and isChar(c1)):
        return True
    if(c2.isupper() and isChar(c2)):
        return True
    return False

def get_number(strText):
    number = 0 
    for i in range(len(strText[2])):
        if(isChar(strText[2][i]) or isChar(strText[3][i])):
            number += 1    
        if(isChar(strText[4][i])):
            number += 1    
        if(isChar(strText[5][i]) or isChar(strText[6][i])):
            number += 1     
    return number

def get5pInfo(strText):
    has_5p = False
    s_5p = 0
    f_5p = 0
    counter = 0
    for i in range(len(strText[2])):
        if(isChar(strText[2][i]) or isChar(strText[3][i])):
            counter += 1            
            if(isMature(strText[2][i] ,strText[3][i])):
                if(not has_5p):
                    has_5p = True
                    s_5p = counter
                else:
                    continue
            elif(has_5p):
                f_5p = counter - 1 
                break
    return [has_5p, s_5p, f_5p]

def get3pInfo(strText, number):
    has_3p = False
    s_3p = 0
    f_3p = 0
    counter = 0
    for i in range(len(strText[6])):
        if(isChar(strText[6][i]) or isChar(strText[5][i])):
            counter += 1            
            if(isMature(strText[6][i] ,strText[5][i])):
                if(not has_3p):
                    has_3p = True
                    s_3p = counter
                else:
                    continue
            elif(has_3p):
                f_3p = counter - 1 
                break        
    return [has_3p, number - f_3p + 1, number - s_3p + 1]    

def convertor(strText):
    index  = []
    values = []
    nucludid = []
    def add(i, v, n):        
        index.append(i)
        values.append(v)
        nucludid.append(n)
            
    f_counter = 1
    r_counter = 0
                    
    number = get_number(strText)    
    [FO, FI, Mid, RI, RO] = strText[2:]
    # main loop    
    for i in range(len(Mid)):           
        
        if(isChar(FO[i]) and FI[i] == " " and Mid[i] == " " and RI[i] == " "  and RO[i] == "-" ):
            add(f_counter, 0, FO[i])                
            f_counter += 1                      
                
        elif(FO[i] == "-" and FI[i] == " " and Mid[i] == " " and RI[i] == " "  and isChar(RO[i])):                                            
            add(number - r_counter, 0, RO[i])                
            r_counter += 1                      
        
        elif(isChar(FO[i]) and not isChar(FI[i]) and not isChar(Mid[i]) and not isChar(RI[i]) and isChar(RO[i])):
            add(f_counter, 0, FO[i])                    
            add(number - r_counter, 0, RO[i])                                
            f_counter += 1        
            r_counter += 1                              
            
        elif(not isChar(FO[i]) and isChar(FI[i]) and Mid[i] == "|" and isChar(RI[i]) and not isChar(RO[i])):        
            add(f_counter, number - r_counter, FI[i])                        
            add(number - r_counter, f_counter, RI[i])                
            f_counter += 1        
            r_counter += 1                  
            
        elif(not isChar(FO[i]) and isChar(FI[i]) and isChar(Mid[i]) and isChar(RI[i]) and not isChar(RO[i])):
            add(f_counter, 0, FI[i])
            add(f_counter + 1, 0, Mid[i])
            add(number - r_counter, 0, RI[i])
            f_counter += 2
            r_counter += 1
            
        elif(not isChar(FO[i]) and isChar(FI[i]) and Mid[i] == " " and isChar(RI[i]) and not isChar(RO[i])):                
            add(f_counter, 0, FI[i])                    
            add(number - r_counter, 0, RI[i])                    
            f_counter += 1        
            r_counter += 1                  
            
        elif(not isChar(FO[i]) and not isChar(FI[i]) and isChar(Mid[i]) and not isChar(RI[i]) and not isChar(RO[i])):                
            add(f_counter, 0, Mid[i])                                
            f_counter += 1                    

    # sort indexs and other base on indexes
    inds = np.array(index).argsort()    
    nucludid = np.array(nucludid)[inds]       
    return ''.join(nucludid)

# Download data from Mirbase

In [7]:
directory = './miRBase'
base = "https://www.mirbase.org/ftp/CURRENT"        

In [8]:
mature = fasta_to_df(f'{directory}/mature.fa')
mature_high_conf = fasta_to_df(f'{directory}/mature_high_conf.fa')
mature['trim tag'] = mature['tag'].apply(lambda line: ' '.join(line.split(' ')[:2]))
mature['confidence'] = mature['trim tag'].isin(mature_high_conf['tag'])

In [9]:
mature['organism'] = mature['tag'].apply(lambda x: x[:3])
print(mature.shape)
mature.head(2)

(48885, 5)


,tag,data,trim tag,confidence,organism
0,cel-let-7-5p MIMAT0000001 Caenorhabditis elega...,UGAGGUAGUAGGUUGUAUAGUU,cel-let-7-5p MIMAT0000001,True,cel
1,cel-let-7-3p MIMAT0015091 Caenorhabditis elega...,CUAUGCAAUUUUCUACCUUACC,cel-let-7-3p MIMAT0015091,True,cel


In [10]:
organism = pd.read_csv(f'./{directory}/organisms.txt',sep='\t')
organism.columns = [c.replace('#','') for c in organism.columns] # remove sharp from columns
print(organism.shape)
organism.head(2)

(285, 5)


,organism,division,name,tree,NCBI-taxid
0,aqu,AQU,Amphimedon queenslandica,Metazoa;Porifera;,400682
1,nve,NVE,Nematostella vectensis,Metazoa;Cnidaria;,45351


In [11]:
items = list(organism['tree'].unique())
items.sort(key=len)
items

['Viruses;',
 'Mycetozoa;',
 'Alveolata;',
 'Metazoa;Porifera;',
 'Metazoa;Cnidaria;',
 'Viridiplantae;Chlorophyta;',
 'Viridiplantae;Embryophyta;',
 'Viridiplantae;Coniferophyta;',
 'Viridiplantae;Magnoliophyta;',
 'Metazoa;Bilateria;Deuterostoma;',
 'Chromalveolata;Heterokontophyta;',
 'Metazoa;Bilateria;Ecdysozoa;Nematoda;',
 'Metazoa;Bilateria;Lophotrochozoa;Annelida;',
 'Metazoa;Bilateria;Lophotrochozoa;Nemertea;',
 'Metazoa;Bilateria;Lophotrochozoa;Mollusca;',
 'Viridiplantae;Magnoliophyta;monocotyledons;',
 'Metazoa;Bilateria;Deuterostoma;Hemichordata;',
 'Metazoa;Bilateria;Deuterostoma;Echinodermata;',
 'Metazoa;Bilateria;Lophotrochozoa;Brachiopoda;',
 'Metazoa;Bilateria;Ecdysozoa;Arthropoda;Hexapoda;',
 'Metazoa;Bilateria;Ecdysozoa;Arthropoda;Crustacea;',
 'Metazoa;Bilateria;Lophotrochozoa;Platyhelminthes;',
 'Metazoa;Bilateria;Ecdysozoa;Arthropoda;Chelicerata;',
 'Metazoa;Bilateria;Ecdysozoa;Arthropoda;Mandibulata;',
 'Viridiplantae;Magnoliophyta;eudicotyledons;Poaceae;',
 'M

In [12]:
selectedTree = organism[organism['tree'].apply(lambda x: "Viridiplantae;" in x)]
print(selectedTree.shape)
selectedTree.head(5)

(86, 5)


,organism,division,name,tree,NCBI-taxid
66,cre,CRE,Chlamydomonas reinhardtii,Viridiplantae;Chlorophyta;,3055
67,pta,PTA,Pinus taeda,Viridiplantae;Coniferophyta;,3352
68,ppt,PPT,Physcomitrella patens,Viridiplantae;Embryophyta;,3218
69,smo,SMO,Selaginella moellendorffii,Viridiplantae;Embryophyta;,88036
70,ath,ATH,Arabidopsis thaliana,Viridiplantae;Magnoliophyta;eudicotyledons;Bra...,3702


In [15]:
selectedTree = selectedTree[selectedTree['name'] == "Triticum aestivum"]

In [16]:
selected = mature[mature['organism'].isin(selectedTree['organism'])]
print(selected.shape)
selected.head(1)

(125, 5)


,tag,data,trim tag,confidence,organism
6755,tae-miR159a MIMAT0005343 Triticum aestivum miR...,UUUGGAUUGAAGGGAGCUCUG,tae-miR159a MIMAT0005343,True,tae


In [17]:
with open(f'./{directory}/miRNA.str') as file:
    strRNA = file.read()
    lines = strRNA.split('\n')
nuclutide = []
hit_start = []
hit_end = []
for i in tqdm(range(0,len(lines), 8)):
    tag = lines[i]
    if(tag[1:4] == "tae"):
        strText = lines[i:(i+7)]        
        if(len(strText) == 7):
            nuc = convertor(strText).lower()            
            number = get_number(strText)    
            [has_5p, s_5p, f_5p] = get5pInfo(strText)                
            [has_3p, s_3p, f_3p] = get3pInfo(strText, number)
            if(has_5p):
                if(s_5p > f_5p):                    
                    continue
            if(has_3p):
                if(s_3p > f_3p):                    
                    continue
            if(has_5p):
                nuclutide.append(nuc)
                hit_start.append(s_5p)                
                hit_end.append(f_5p)
            if(has_3p):
                nuclutide.append(nuc)                
                hit_start.append(s_3p)
                hit_end.append(f_3p)                                                      
precursor = pd.DataFrame({"data": nuclutide, "hit_start": hit_start, "hit_end": hit_end})
precursor['hit'] = precursor.apply(lambda row: row['data'][(row['hit_start']-1):(row['hit_end'])].upper(), axis=1)
precursor['confidence'] = precursor['hit'].apply(lambda hit: selected['confidence'][selected['data'] == hit].sum() > 0)
precursor['tag'] = precursor.index
print(precursor.shape)
precursor.head()

  0%|          | 0/38590 [00:00<?, ?it/s]

(123, 6)


,data,hit_start,hit_end,hit,confidence,tag
0,guggagcuccuaucauuccaaugaagggucuaccgaaggguuugug...,154,174,UUUGGAUUGAAGGGAGCUCUG,True,0
1,gagcuccuuucgguccaaaaagggguguugcugugggucgauugag...,230,250,UUUGGAUUGAAGGGAGCUCUG,True,1
2,agaggugaaaacaaugggauaugccuggcucccuguaugccacucg...,22,42,UGCCUGGCUCCCUGUAUGCCA,False,2
3,gguggagaagcagggcacgugcauccauuuccagcucggcauuccc...,3,23,UGGAGAAGCAGGGCACGUGCA,False,3
4,cugcccaagggaacgagugaagcugccagcaugaucuagcuccgag...,18,38,UGAAGCUGCCAGCAUGAUCUA,False,4


In [18]:
precursor

,data,hit_start,hit_end,hit,confidence,tag
0,guggagcuccuaucauuccaaugaagggucuaccgaaggguuugug...,154,174,UUUGGAUUGAAGGGAGCUCUG,True,0
1,gagcuccuuucgguccaaaaagggguguugcugugggucgauugag...,230,250,UUUGGAUUGAAGGGAGCUCUG,True,1
2,agaggugaaaacaaugggauaugccuggcucccuguaugccacucg...,22,42,UGCCUGGCUCCCUGUAUGCCA,False,2
3,gguggagaagcagggcacgugcauccauuuccagcucggcauuccc...,3,23,UGGAGAAGCAGGGCACGUGCA,False,3
4,cugcccaagggaacgagugaagcugccagcaugaucuagcuccgag...,18,38,UGAAGCUGCCAGCAUGAUCUA,False,4
...,...,...,...,...,...,...
118,cggacgcagauucagugacauucccccgugacauucucggugacau...,43,60,ACAUUCAGUCAUUGACAU,False,118
119,uauaucuagacauagacuuuguucagauaauaguuacucccuccgu...,57,74,AAGAGAUUUUGAAGGGAU,False,119
120,cuacaugacaucgugacuucgcgguuuuagcuguaggacaccguau...,71,88,UGUCAAAAGUUGGAUAUU,False,120
121,guuaguuuaguuucacaaaaccacacuauugaugaauggauucacu...,71,90,CUUGGAUGAGAACAUGGCAU,False,121


In [19]:
precursor.to_csv(f'{temp_path}/precursor_info.csv')
df_to_fasta(precursor, f'{temp_path}/mature_microRNA_queries.fasta')

# Remove redundant

## cdhit-est

In [20]:
!./software/cdhit/cd-hit-est -i ./{temp_path}/mature_microRNA_queries.fasta  -o ./{temp_path}/NR_mature_microRNA_queries.fasta \
    -c 1 -r 0 -G 1 -g 1 -b 30 -l 10 -aL 0 -AL 99999999 -aS 0 \
    -AS 99999999 -s 0 -S 0

Program: CD-HIT, V4.8.1 (+OpenMP), Oct 23 2021, 21:45:39
Command: ./software/cdhit/cd-hit-est -i
         ././Experiment/T.aestivum/positive/Temp/mature_microRNA_queries.fasta
         -o
         ././Experiment/T.aestivum/positive/Temp/NR_mature_microRNA_queries.fasta
         -c 1 -r 0 -G 1 -g 1 -b 30 -l 10 -aL 0 -AL 99999999 -aS
         0 -AS 99999999 -s 0 -S 0

Started: Sat Dec  3 22:27:01 2022
                            Output                              
----------------------------------------------------------------
total seq: 123
longest and shortest : 426 and 54
Total letters: 16541
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 0M
Buffer          : 1 X 12M = 12M
Table           : 1 X 16M = 16M
Miscellaneous   : 0M
Total           : 28M

Table limit with the given memory limit:
Max number of representatives: 3247413
Max number of word counting entries: 96377554

comparing sequences from          0  to        123

      123  finished 

## reformat

In [21]:
with open(f'./{temp_path}/NR_mature_microRNA_queries.fasta.clstr','r') as file:
    text = file.read()
lines = [line for line in text.split('\n') if len(line) > 0]
cluster = []
seqid = []
last_cluster = ""
for l in lines:
    if(l[0]=='>'):        
        last_cluster = l.replace('>Cluster ',"C")
    else:        
        cluster.append(last_cluster)
        seqid.append(l.split(', >')[1].split('...')[0])                
seq2cluster = pd.DataFrame({'seqid': seqid,'cluster': cluster})
print(seq2cluster.shape)
seq2cluster.head(2)    

(123, 2)


,seqid,cluster
0,8,C0
1,84,C1


In [22]:
df = fasta_to_df(f"./{temp_path}/mature_microRNA_queries.fasta")
seq2cluster = pd.merge(df,seq2cluster,how="inner",left_on='tag',right_on="seqid")
precursor['tag'] = precursor['tag'].apply(lambda x : str(x))
seq2cluster = pd.merge(seq2cluster, precursor,how="inner",left_on='seqid',right_on="tag")[['cluster','seqid','tag_x', 'confidence']]
seq2cluster.columns = ['cluster','seqid','tag', 'confidence']
print(seq2cluster.shape)
display(seq2cluster.head(2))
seq2cluster.to_csv(f'./{temp_path}/seq2cluster.csv',index=False)

(123, 4)


,cluster,seqid,tag,confidence
0,C19,0,0,True
1,C7,1,1,True


In [23]:
# todo: sorted first by cluster then by seqid
seq2cluster.sort_values("cluster").head(2)

,cluster,seqid,tag,confidence
8,C0,8,8,False
84,C1,84,84,False


In [24]:
df = fasta_to_df(f"./{temp_path}/NR_mature_microRNA_queries.fasta")
df['tag'] = df['tag'].apply(lambda x : str(x))
df = pd.merge(df,seq2cluster,how="inner",left_on='tag',right_on="seqid")[['cluster','data']]

lines = []
df.apply(lambda row: lines.append(f">{row['cluster']}\n{row['data']}\n"),axis=1)
print(df.shape)
with open(f'./{temp_path}/BLASTn_queries.fasta','w') as file:
    file.write(''.join(lines))

(120, 2)


# BlastN

In [25]:
!makeblastdb -in {input_genome_path} -dbtype nucl -out ./{temp_path}/blastn_database



Building a new DB, current time: 12/03/2022 22:27:20
New DB name:   /home/jupyter/plant_microRNA_prediction/Experiment/T.aestivum/positive/Temp/blastn_database
New DB title:  ./Experiment/T.aestivum/GCF_018294505.1_IWGSC_CS_RefSeq_v2.1_genomic.fna
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 91589 sequences in 303.709 seconds.


In [26]:
header = 'qseqid sseqid qstart qend sstart send qseq sseq evalue bitscore score length pident nident mismatch positive gapopen gaps ppos frames qframe sframe sstrand qcovs qcovhsp qlen slen'

In [27]:
!blastn -query ./{temp_path}/BLASTn_queries.fasta \
        -out ./{temp_path}/BLASTn_result \
        -num_threads {mp.cpu_count()} \
        -db ./{temp_path}/blastn_database \
        -word_size 7 \
        -penalty -3 \
        -reward 2 \
        -gapopen 5 \
        -gapextend 2 \
        -outfmt '6 qseqid sseqid qstart qend sstart send qseq sseq evalue bitscore score length pident nident mismatch positive gapopen gaps ppos frames qframe sframe sstrand qcovs qcovhsp qlen slen'       

In [28]:
df_blastn = pd.read_csv(f'./{temp_path}/BLASTn_result', sep='\t',header=None)
df_blastn.columns = header.replace("  "," ").split(" ")
print(df_blastn.shape)
df_blastn.head(2)

(2711698, 27)


,qseqid,sseqid,qstart,qend,sstart,send,qseq,sseq,evalue,bitscore,...,gaps,ppos,frames,qframe,sframe,sstrand,qcovs,qcovhsp,qlen,slen
0,C19,NC_057802.1,1,177,6525140,6524964,GTGGAGCTCCTATCATTCCAATGAAGGGTCTACCGAAGGGTTTGTG...,GTGGAGCTCCTATCATTCCAATGAAGGGTCTACCGAAGGGTTTGTG...,7.030000e-85,320.0,...,0,100.00,1/-1,1,-1,minus,100,100,177,619618552
1,C19,NC_057801.1,1,177,12799326,12799150,GTGGAGCTCCTATCATTCCAATGAAGGGTCTACCG-AAGGGTTTGT...,GTGGAGCTCCTATCATTCCAATGAAGGGTCTACCGGAAGGGTTTGT...,4.150000e-75,288.0,...,2,96.63,1/-1,1,-1,minus,100,100,177,851934019


In [29]:
threshold = 0
df_blastn['Nonconformity'] = df_blastn['qlen'] - (abs(df_blastn['qend'] - df_blastn['qstart']) + 1) + df_blastn['gaps'] + df_blastn['mismatch']
df_blastn = df_blastn[df_blastn['Nonconformity'] <= threshold]
print(df_blastn.shape)
df_blastn.head(2)

(277, 28)


,qseqid,sseqid,qstart,qend,sstart,send,qseq,sseq,evalue,bitscore,...,ppos,frames,qframe,sframe,sstrand,qcovs,qcovhsp,qlen,slen,Nonconformity
0,C19,NC_057802.1,1,177,6525140,6524964,GTGGAGCTCCTATCATTCCAATGAAGGGTCTACCGAAGGGTTTGTG...,GTGGAGCTCCTATCATTCCAATGAAGGGTCTACCGAAGGGTTTGTG...,7.030000e-85,320.0,...,100.0,1/-1,1,-1,minus,100,100,177,619618552,0
30,C33,NC_057807.1,1,147,601833629,601833775,AGAGGTGAAAACAATGGGATATGCCTGGCTCCCTGTATGCCACTCG...,AGAGGTGAAAACAATGGGATATGCCTGGCTCCCTGTATGCCACTCG...,1.080000e-68,266.0,...,100.0,1/1,1,1,plus,100,100,147,714697677,0


In [30]:
# alignment length adjustment
def blastn_adjust(row):
    if(row['sstrand'] == "plus"):        
        row['sstart'] = max(1, row['sstart'] - (row['qstart'] - 1))
        row['send'] = min(row['slen'], row['send'] + (row['qlen'] - row['qend']))
    if(row['sstrand'] == "minus"):        
        row['send'] = max(1, row['send'] - (row['qstart'] - 1 ))
        row['sstart'] = min(row['slen'], row['sstart'] + (row['qlen'] - row['qend'] ))
    return row
    
df_blastn = df_blastn.apply(lambda row: blastn_adjust(row), axis=1)

In [31]:
# remore redundancy and hold best one base of Nonconformity value
df_blastn = df_blastn.sort_values(["Nonconformity", "evalue"], ascending = (True, True))
df_blastn = df_blastn.drop_duplicates(subset=['sseqid','sstart', 'qseqid', 'send','sstrand'], keep='first')
df_blastn.to_csv(f'./{temp_path}/filtered_out_blastn.csv')
print(df_blastn.shape)

(277, 28)


In [32]:
df_blastn = pd.merge(df_blastn, seq2cluster, how='inner', left_on='qseqid', right_on='cluster')
df_blastn = pd.merge(df_blastn, precursor, how='inner', left_on='seqid', right_on='tag')
print(df_blastn.shape)
df_blastn.head()

(306, 38)


,qseqid,sseqid,qstart,qend,sstart,send,qseq,sseq,evalue,bitscore,...,cluster,seqid,tag_x,confidence_x,data,hit_start,hit_end,hit,confidence_y,tag_y
0,C1,NC_057802.1,1,344,577914993,577915336,TGCATTACATATAAAACCTTCAGCTATCCATCTCCATATCAGCCTG...,TGCATTACATATAAAACCTTCAGCTATCCATCTCCATATCAGCCTG...,3.320000e-175,621.0,...,C1,84,84,False,ugcauuacauauaaaaccuucagcuauccaucuccauaucagccug...,11,32,AUAAAACCUUCAGCUAUCCAUC,False,84
1,C3,NC_057807.1,1,295,168546905,168547199,ATAAGTACTCCCTCTGTTCCTAAATACTCCCTCCGTCCCAAAAATT...,ATAAGTACTCCCTCTGTTCCTAAATACTCCCTCCGTCCCAAAAATT...,1.120000e-148,533.0,...,C3,13,13,False,auaaguacucccucuguuccuaaauacucccuccgucccaaaaauu...,96,115,UAGAUACAUCCGUAUCUAGA,False,13
2,C3,NC_057807.1,1,295,332816689,332816395,ATAAGTACTCCCTCTGTTCCTAAATACTCCCTCCGTCCCAAAAATT...,ATAAGTACTCCCTCTGTTCCTAAATACTCCCTCCGTCCCAAAAATT...,1.120000e-148,533.0,...,C3,13,13,False,auaaguacucccucuguuccuaaauacucccuccgucccaaaaauu...,96,115,UAGAUACAUCCGUAUCUAGA,False,13
3,C3,NC_057804.1,1,295,590714470,590714764,ATAAGTACTCCCTCTGTTCCTAAATACTCCCTCCGTCCCAAAAATT...,ATAAGTACTCCCTCTGTTCCTAAATACTCCCTCCGTCCCAAAAATT...,1.120000e-148,533.0,...,C3,13,13,False,auaaguacucccucuguuccuaaauacucccuccgucccaaaaauu...,96,115,UAGAUACAUCCGUAUCUAGA,False,13
4,C4,NC_057804.1,1,280,635694439,635694160,CGGCCGCCATTCAGCCAGAAGAAGCAGCGAGCCAGCGGAGACCGGC...,CGGCCGCCATTCAGCCAGAAGAAGCAGCGAGCCAGCGGAGACCGGC...,1.460000e-140,506.0,...,C4,19,19,False,cggccgccauucagccagaagaagcagcgagccagcggagaccggc...,25,48,CAGCGAGCCAGCGGAGACCGGCAG,False,19


# Result of the blastn to bed file

In [33]:
flanking_value = 20
df = df_blastn[['qseqid', 'sseqid', 'sstart', 'send', 'sstrand','slen', 'hit_start', 'hit_end']]
df['ones'] = 1
df.head(2)

<ipython-input-33-3cfdabc28c3c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ones'] = 1


,qseqid,sseqid,sstart,send,sstrand,slen,hit_start,hit_end,ones
0,C1,NC_057802.1,577914993,577915336,plus,619618552,11,32,1
1,C3,NC_057807.1,168546905,168547199,plus,714697677,96,115,1


In [34]:
def switch(row):
    if(row['sstart'] > row['send']):        
        temp = row['sstart']
        row['sstart'] = row['send']
        row['send'] = temp
    return row
df = df.apply(lambda row: switch(row), axis=1)

In [35]:
def convert(inp):
    if(inp == "plus"):
        return "forward"
    if(inp == "minus"):
        return "reverse"
    raise Exception('Error, sstrand contains illegal word! only "plus" and "minus" are allowed')
df['strand'] = df['sstrand'].apply(lambda x: convert(x))

In [36]:
def convert2sign(inp):
    if(inp == "plus"):
        return "+"
    if(inp == "minus"):
        return "-"
    raise Exception('Error, sstrand contains illegal word! only "plus" and "minus" are allowed')
df['sign'] = df['sstrand'].apply(lambda x: convert2sign(x))

In [37]:
df['hit_length'] = df.apply(lambda row: abs(row['send'] - row['sstart']) + 1 ,axis=1)

## convert sstart and send from location to index (range)

In [38]:
df['sstart'] = df['sstart'].apply(lambda x: x - 1)

In [39]:
df['downstream_flanking'] = df['sstart'].apply(lambda x: flanking_value if x > flanking_value else x)

In [40]:
df['upstream_flanking'] = df.apply(lambda row: flanking_value if (row['send']+flanking_value) <= row['slen'] else row['slen'] - row['send'],axis=1)

In [41]:
df['hit_start'] = df.apply(lambda row: row['downstream_flanking'] + row['hit_start'] -1 if row['sign'] == "+" else row['upstream_flanking'] + row['hit_start'] -1, axis=1)
df['hit_end'] = df.apply(lambda row: row['downstream_flanking'] + row['hit_end'] if row['sign'] == "+" else row['upstream_flanking'] + row['hit_end'],axis=1)

In [42]:
df['sstart'] = df['sstart'].apply(lambda x: max(x - flanking_value, 0))
df['send'] = df.apply(lambda row: min(row['send'] + flanking_value , row['slen']),axis=1)

In [43]:
df['tag'] = df.apply(lambda row: f">{row['sseqid']}:{row['sstart']}-{row['send']}({row['sign']})",axis=1)
df['reformated_tag'] = df['tag'].apply(lambda t: reformat(t))
df[['tag', 'reformated_tag', 'hit_start', 'hit_end']].to_csv(f'./{temp_path}/hit_index_info.csv')#, index=False)

In [44]:
df['location_tag'] = df.apply(lambda row: f">{row['sseqid']}|{row['sign']}|{row['sstart'] + 1}-{row['send']}|{row['hit_start']+1}-{row['hit_end']}",axis=1)
df[['location_tag','qseqid']].to_csv(f'./{temp_path}/pipe_seprated_location_list.csv',index=False,sep='\t')

In [45]:
df[['sseqid','sstart','send','strand','ones', 'sign']].to_csv(f'./{temp_path}/extension_index.bed', 
        index=False, header=False, sep="\t")

# Extention

In [46]:
!bedtools getfasta -fi {input_genome_path} -fo ./{temp_path}/extended_original.txt -s -bed ./{temp_path}/extension_index.bed
!rm input_genome.fna.fai

rm: cannot remove 'input_genome.fna.fai': No such file or directory


# Convert hit region to upper case and other region to lower case

In [47]:
ext = fasta_to_df(f'./{temp_path}/extended_original.txt')
info = pd.read_csv(f'./{temp_path}/hit_index_info.csv')
info['tag'] = info['tag'].apply(lambda x: x[1:])
print(info.shape)
info.head(2)

(306, 5)


,Unnamed: 0,tag,reformated_tag,hit_start,hit_end
0,0,NC_057802.1:577914972-577915356(+),>NC_0578021_577914972-577915356_+_,30,52
1,1,NC_057807.1:168546884-168547219(+),>NC_0578071_168546884-168547219_+_,115,135


In [48]:
ext = ext.sort_values(by=['tag']).reset_index()
ext['help_tag'] = ext.apply(lambda r: r['tag'] + str(r.name),axis=1)
del ext['tag']

info = info.sort_values(by=['tag']).reset_index()
info['help_tag'] = info.apply(lambda row: row['tag']+ str(row.name),axis=1)
def redefined_tag(row):
    tag = row['tag']
    [sstart, send] = tag.split(':')[-1].split('(')[0].split('-')
    sstart = int(sstart) + 1
    sign = tag.split('(')[-1].split(')')[0]    
    return f"{tag.split(':')[0]}|{sign}|{sstart}-{send}|{row['hit_start']+1}-{row['hit_end']}"
info['tag'] = info.apply(lambda row: redefined_tag(row),axis=1)
ext = pd.merge(ext,info,how='inner', on='help_tag')

def emphasis_hit(row):
    seq = list(row['data'].lower())            
    s = row['hit_start']
    e = row['hit_end']
    seq[s:e] = list(''.join(seq[s:e]).upper())    
    return ''.join(seq)
    
ext['data'] = ext.apply(lambda row: emphasis_hit(row),axis=1)
ext = ext.drop_duplicates(subset=['data'], keep='first')
df_to_fasta(ext[['tag','data']],f"./{temp_path}/extended_modified.txt")

# Protein coding elimination [Diamond]

In [46]:
#!./diamond makedb --in ./NR/nr -d ./Temp/diamond_output

In [47]:
#!./diamond blastx -d ./Temp/diamond_output.dmnd \
#                  -q ./Temp/extended_modified.txt \
#                  -o ./Temp/diamond_matches.tsv \
#                  -p 22

In [48]:
dmn = pd.read_csv(f"./{temp_path}/diamond_matches.tsv", sep='\t', header=None)
dmn.columns = 'qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore'.split(' ')
coding_seq = dmn['qseqid'].unique()

In [49]:
def clear(inp):
    if(inp[:9] == "reverse::"):
        return inp[9:]
    if(inp[:9] == "forward::"):
        return inp[9:]
    return inp
coding_seq = pd.Series(coding_seq).apply(lambda x : clear(x))

ext = fasta_to_df(f'./{temp_path}/extended_modified.txt')
print(f'total:      {ext.shape[0]}')
non_coding = ext[~ext['tag'].isin(coding_seq)]
print(f'non_coding: {non_coding.shape[0]}')
df_to_fasta(non_coding,f'./{temp_path}/extended_modified_non_coding.txt')
coding = ext[ext['tag'].isin(coding_seq)]
print(f'coding:     {coding.shape[0]}')
df_to_fasta(coding,f'./{temp_path}/extended_modified_coding.txt')

total:      26
non_coding: 16
coding:     10


# RNA 2d prediction

## Mfold

In [50]:
counter = 0
base = f"{result_path}/secondary_structure/mfold/"
!rm -r {base}
!mkdir -p {base}
df = fasta_to_df(f'./{temp_path}/extended_modified_non_coding.txt')

for index, row in df.iterrows():    
    tag = reformat(row['tag'])
    if(not os.path.exists(base + tag)):
        os.makedirs(base + tag)            
    with open(base + f"{tag}/SEQ.FASTA",'w') as file:
        file.write(f">{row['tag']}\n{row['data']}")
    counter += 1    
    #if(counter >= 100):
      #  break

rm: cannot remove './Experiment/H.vulgare/positive/Result/secondary_structure/mfold/': No such file or directory


In [51]:
%%capture
remove_lock = False
def run_mfold(tag):
    tag = reformat(tag)
    %cd {base + tag}
    !mfold  SEQ="SEQ.FASTA" T=22
    if(not remove_lock):
        !find . -name "SEQ*" -not -name "*.ct" -not -name "*.pdf" -not -name "*SEQ.FASTA" -not -type d -delete
    %cd {current_path}

if __name__ == '__main__':        
    pool = mp.Pool(mp.cpu_count() - 3)
    pool.map(run_mfold, df['tag'])  

## Mxfold2

In [ ]:
#!wget https://github.com/keio-bioinformatics/mxfold2/releases/download/v0.1.1/mxfold2-0.1.1.tar.gz
#!pip3 install mxfold2-0.1.1.tar.gz
#!rm mxfold2-0.1.1.tar.gz

In [ ]:
!mxfold2 predict ./extended.txt > Result/secondary_structure/mxfold2_result.txt

In [ ]:
df = fasta_to_df('./Result/secondary_structure/mxfold2_result.txt')
df = df.apply(lambda row: bracket_row(row) , axis=1)
df.head(2)

In [ ]:
base = "./Result/secondary_structure/mxfold2/"
!rm -r {base}
!mkdir -p {base}
for index, row in df.iterrows():    
    if(not os.path.exists(base + reformat(row['tag']))):
        os.makedirs(base + reformat(row['tag']))        
    tag = reformat(row['tag'])
    with open(base + f"{tag}/{tag}.ct",'w') as file:
        bracket = row['bracket'].split(' ')[0]
        deltaG = row['bracket'].split(' ')[1]
        ct = bracket_to_ct(row['tag'], row['data'], bracket, deltaG)
        file.write(ct)    

## Vienna package

In [ ]:
#!wget https://www.tbi.univie.ac.at/RNA/download/ubuntu/ubuntu_20_04/viennarna_2.4.18-1_amd64.deb -O viennarna.deb
#!sudo dpkg -i ./viennarna.deb
#!sudo apt-get -f install
#!rm viennarna.deb

In [96]:
base = f"{result_path}/secondary_structure/viennarna/"
!rm -r {base}
!rm {result_path}/secondary_structure/viennarna_result.txt
!mkdir -p {base}

rm: cannot remove 'Experiment/A.thaliana/Result/secondary_structure/viennarna/': No such file or directory
rm: cannot remove 'Experiment/A.thaliana/Result/secondary_structure/viennarna_result.txt': No such file or directory


In [102]:
#%cd {base}
!RNAfold --jobs=0 --infile {current_path}/{temp_path}/extended_modified.txt  --noPS -T 22 > {current_path}/{base}/viennarna_result.txt
#%cd {current_path}

In [107]:
df = fasta_to_df(f'{result_path}/secondary_structure/viennarna/viennarna_result.txt')
df = df.apply(lambda row: bracket_row(row) , axis=1)
print(df.shape)
df.head(2)

(17793, 3)


,tag,data,bracket
0,NC_000932.1|+|101567-101986|201-220,aaucccagggcucaacccuggacaggcgguggaaacuaccaagcuu...,.(((((((((.....)))))).((((((((((...)))))..))))...
1,NC_000932.1|+|105092-105512|201-221,aaucccgugugaaucagcaaggaccaccuugcaaggcuaaauacuc...,.....(((((......((((((....))))))....(((..(((((...


In [108]:
for index, row in df.iterrows():    
    tag = reformat(row['tag'])
    if(not os.path.exists(base + tag)):
        os.makedirs(base + tag)      
    with open(base + f"{tag}/{tag}.ct",'w') as file:
        bracket = row['bracket'].split(' ')[0]
        deltaG = row['bracket'].split(' ')[1]
        ct = bracket_to_ct(row['tag'], row['data'], bracket, deltaG, False)
        file.write(ct)    

In [109]:
import glob
for file in glob.glob(f"{base}*.ps"):    
    f = file[len(base):-6] # _ss.ps 
    f = reformat(f)        
    shutil.move(file, f"{base}{f}/{f}.ps")    

## ContraFold

In [ ]:
#!wget http://contra.stanford.edu/contrafold/contrafold_v2_02.tar.gz
#!tar -xvzf contrafold_v2_02.tar.gz && rm contrafold_v2_02.tar.gz
#%cd contrafold/src
#!make clean
#!make 
# to file must changed to be complieable # utility.hpp and optimization.c++ files

In [ ]:
counter = 0
base = f"./{result_path}/secondary_structure/contrafold/"
!rm -r {base}
!mkdir -p {base}
df = fasta_to_df(f'./{temp_path}/extended.txt')

for index, row in tqdm(df.iterrows()):    
    tag = reformat(row['tag'])
    if(not os.path.exists(base + tag)):
        os.makedirs(base + tag)            
    with open(base + f"{tag}/{tag}.FASTA",'w') as file:
        file.write(f">{row['tag']}\n{row['data']}")
    counter += 1        

In [ ]:
def run_contrafold(tag):
    tag = reformat(tag)    
    %cd Software/contrafold/src
    !./contrafold predict ../..{base[1:]}{tag}/{tag}.FASTA > ../..{base[1:]}{tag}/{tag}.dot
    with open(f"../..{base[1:]}{tag}/{tag}.dot", 'r') as file:
        text = file.read()
    text = [l for l in text.split("\n") if l[:len(">structure")] != ">structure"]    
    header = text[0]
    with open(f"../..{base[1:]}{tag}/{tag}.dot", 'w') as file:
        file.write('\n'.join(text[1:]))    
    !RNAeval  ../..{base[1:]}{tag}/{tag}.dot -T 20 > ../..{base[1:]}{tag}/{tag}.dotdg    
    with open(f"../..{base[1:]}{tag}/{tag}.dotdg", 'r') as file:
        text = file.read()
    with open(f"../..{base[1:]}{tag}/{tag}.dot", 'w') as file:
        file.write(header + "\n" + text)    
    
    df = fasta_to_df(f'../..{base[1:]}{tag}/{tag}.dot')
    df = df.apply(lambda row: bracket_row(row) , axis=1)        
    tag = reformat(df['tag'][0])
    with open(f'../..{base[1:]}{tag}/{tag}.ct','w') as file:
        bracket = df['bracket'][0].split(' ')[0]        
        deltaG = df['bracket'][0].split(' ')[1]
        ct = bracket_to_ct(df['tag'][0], df['data'][0], bracket, deltaG, False)
        file.write(ct)    
    #!rm ../..{base[1:]}{tag}/{tag}.dot
    #!rm ../..{base[1:]}{tag}/{tag}.dotdg
    !rm ../..{base[1:]}{tag}/{tag}.FASTA
    %cd {current_path}        

if __name__ == '__main__':        
    pool = mp.Pool(mp.cpu_count() - 1)  
    pool.map(run_contrafold, df['tag'].iloc[:10])

In [ ]:
'''path = 'secondary_structure/contrafold/AMWY020333941_469-893_-_/AMWY020333941_469-893_-_.dot'
!RNAeval  {path} -T 20 -v'''; 

# CTAnalizer

In [21]:
# only select those not ran before
ss_method = "mfold"
base = f"{result_path}/secondary_structure/{ss_method}/"
df = fasta_to_df(f'./{temp_path}/extended_modified_non_coding.txt')
index_list =[]
for index, row in df.iterrows():
    tag = reformat(row['tag']) 
    if(len(glob.glob(f'{base + tag}/*.ct')) != 0):
        index_list.append(index)
df = df.iloc[index_list,:]
print(df.shape)

(204, 2)


In [22]:
def run(tag, path, extra):             
    return get_row(tag, path,extra)
    try:
        return get_row(tag, path,extra)
    except Exception as e:
        print(str(e), tag)                
        return pd.Series()
        
def get_df_by_tag(tag , extra=0):           
    ct_files = glob.glob(f'{base}{reformat(tag)}/*.ct')    
    ct_files = [i for i in ct_files if i[-6:] != 'SEQ.ct']
    return pd.Series(ct_files).apply(lambda path: run(tag, path,extra))    

# Apply on current data

In [23]:
seq2cluster = pd.read_csv(f"{temp_path}/seq2cluster.csv")
seq2cluster['tag'] = seq2cluster['tag'].apply(lambda x: str(x))
seq2cluster['tag'] = seq2cluster.groupby(['cluster'])['tag'].transform(lambda x: ','.join(x))
seq2cluster['seqid'] = seq2cluster['seqid'].apply(lambda x: str(x))
seq2cluster['seqid'] = seq2cluster.groupby(['cluster'])['seqid'].transform(lambda x: ','.join(x))
seq2cluster = seq2cluster.drop_duplicates()
tag2cluster = pd.read_csv(f'./{temp_path}/pipe_seprated_location_list.csv',sep='\t')
tag2cluster['location_tag'] = tag2cluster['location_tag'].apply(lambda x : x[1:])
data = pd.merge(seq2cluster,tag2cluster,how='inner', left_on='cluster', right_on='qseqid')
data['Reference miRNA cluster'] = data['cluster']
data['Reference miRNA IDs'] = data['seqid']
data['Reference miRNA IDs and species'] = data['tag']
data = data[['location_tag','Reference miRNA cluster', 'Reference miRNA IDs', 'Reference miRNA IDs and species','confidence']]

In [24]:
rcols_ref = ['Reference miRNA cluster',
             'Reference miRNA IDs',
             'Reference miRNA IDs and species']
rcols_boi = ['boi seq', 'boi name', 'boi dotbracket']

rcols = [*rcols_ref,
         *rcols_boi]

rcols_dg = [*rcols, 'delta G']

def selection(row):        
    global repeted
    tuple_row = tuple(row)            
    if(tuple_row not in repeted):
        repeted[tuple_row] = row.name
        return True
    return False

def boi_selection(row):    
    global repeted_boi        
    tuple_row_boi = tuple(row[rcols_boi])
    dg = row['delta G']
    if(tuple_row_boi not in repeted_boi):
        repeted_boi[tuple_row_boi] = { 
            "counter": 1,
            "dg": dg,
            "ref clusters": row['Reference miRNA cluster'],
            "ref ids": row['Reference miRNA IDs'],
            "ref species": row['Reference miRNA IDs and species'],
            "lock": False
        }
    else:            
        value =  repeted_boi[tuple_row_boi]
        value['counter'] += 1
        value['dg'] = min(value['dg'] , dg)
        value['ref clusters'] += "," + row['Reference miRNA cluster']
        value['ref ids'] += "," + row['Reference miRNA IDs']
        value['ref species'] += "," +  row['Reference miRNA IDs and species']                
        repeted_boi[tuple_row_boi] = value        

In [25]:
!rm ./{result_path}/ct_analizer.csv
chunksize = 1 * (10 ** 4)
max_workers = mp.cpu_count() - 4
num_terminal = 5 # acceptable_terminal_structures

repeted = {}
repeted_boi = {}
header = True
orders = None
arr = np.array_split(df['tag'], max(df['tag'].shape[0]//chunksize , 1))
for chunk in tqdm(arr):
    dfs = []
    for row in process_map(get_df_by_tag , chunk, tqdm_class=tqdm, max_workers=max_workers, chunksize=5):        
        dfs.append(row)
    chunk = pd.concat(dfs,axis=0)
    chunk = pd.merge(data, chunk, how='right', left_on = 'location_tag', right_on ='seq name')
    del chunk['location_tag']
    if(header):
        orders = chunk.columns
    for col in orders:
        if(col not in chunk.columns):
            chunk[col] = np.nan            
        
    for col in chunk.columns:
        if(col not in orders ):
            print(f"Error in {col}")        
    chunk = chunk.reindex(columns=orders)
    chunk = chunk.replace(np.nan, '-').replace('', '-')                    
    # delete repeated
    selected = chunk[rcols].apply(lambda row: selection(row), axis=1)
    chunk = chunk[selected]
    
    # cluster refs
    chunk[rcols_dg].apply(lambda row: boi_selection(row), axis=1)        
    chunk.to_csv(f"./{result_path}/ct_analizer.csv", header=header, mode='a', index=False)    
    header = False

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/204 [00:00<?, ?it/s]

In [26]:
!rm ./{result_path}/ct_analizer_clustered.csv
def isKeepCluster(row):
    global repeted_boi    
    dg = row['delta G']    
    if(row["boi name"] == "-"):
        return False
    tuple_row_boi = tuple(row[rcols_boi])    
    value = repeted_boi[tuple_row_boi]
    if(value['counter'] == 1):
        return True
    if(value['dg'] != dg):        
        return False
    if(value['lock']):
        return False
    value['lock'] = True
    repeted_boi[tuple_row_boi] = value
    return True


def makeCluster(row):            
    tuple_row_boi = tuple(row[rcols_boi])    
    value = repeted_boi[tuple_row_boi]
    if(value['counter'] != 1):
        for ref_c in ['ref clusters', 'ref ids', 'ref species']:
            value[ref_c] = value[ref_c].replace(' ,', ',').replace(', ', ',')                        
            value[ref_c] = value[ref_c].split(',')
            value[ref_c] = set(value[ref_c])
            value[ref_c] = ", ".join(value[ref_c])                    
        row['Reference miRNA cluster'] = value["ref clusters"]
        row['Reference miRNA IDs'] = value["ref ids"]
        row['Reference miRNA IDs and species'] = value["ref species"]
    return row


header = True
for chunk in tqdm(pd.read_csv(f"./{result_path}/ct_analizer.csv", chunksize=10 ** 5)):
    chunk = chunk[chunk[rcols_dg].apply(lambda row:  isKeepCluster(row), axis=1)]
    chunk = chunk.apply(lambda row : makeCluster(row), axis=1)
    chunk.to_csv(f"./{result_path}/ct_analizer_clustered.csv", mode='a', index=False)
    header = False

0it [00:00, ?it/s]

# Filters

In [27]:
!rm ./{result_path}/result_level1_filter.csv
filter_run(input_file=  f"./{result_path}/ct_analizer_clustered.csv",
            output_file= f"./{result_path}/result_level1_filter.csv")

1it [00:00,  1.91it/s]


In [28]:
pd.read_csv(f"./{result_path}/result_level1_filter.csv").shape

(625, 183)